# 1.0 本文的研究目的是 <code>情感分析</code>, 通过一些已经定义好的词典来给句子向量化
## 1.1 读取情感词库 boson nlp

In [1]:
def getSentimentWords(url):
    with open(url,'r',encoding='utf8') as f:
        sentiWord = f.readlines()
    eachLine=[i.replace('\n','') for i in sentiWord]
    sentiDict={}
    for line in eachLine:
        twoCom=line.split(" ")
        if len(twoCom)==2:
            sentiDict[twoCom[0]]=float(twoCom[1])
    return sentiDict

BosonWords=getSentimentWords("NLP/Dictionary/BosonNLP/BosonNLP_sentiment_score.txt")
iniLen=len(BosonWords)
print("the length of boson dictionary: ",iniLen)
BosonWords

## 1.2 读取同义词库，因为boson情感词库的词语太少了，难免遇到情感词不够或者匹配不到的情况,需要拓展一下情感词

In [2]:
import numpy as np
import nltk
import json
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
def findSyn(w):
    return [e.lemma_names('cmn') for e in wn.synsets(w, lang='cmn')]

def iniSynonym():
    path = 'NLP/Dictionary/拓展同义词典/cow-not-full.txt'
    words = [l.split('\t')[-1].strip() for l in open(path).readlines()]
    syn_wn = {}
    for w in words:
        try:
            w_ = w.replace('+', '')
            added = []
            syn_lst = findSyn(w)
            for l in syn_lst:
                l_ = l.copy()
                l_.remove(w_)
                added.extend([i.replace('+', '') for i in l_])
            if added:
                syn_wn[w_] = added
        except:
            pass

    with open('NLP/Dictionary/拓展同义词典/wordnetSyn.json', 'w') as f:
        json.dump(syn_wn, f)

    for k,v in syn_wn.items():
        syn_wn[k]=list(np.unique(v))

    print("the length of Synonym dictionary: ",len(syn_wn))
    print("the increased number of words in Boson: ", len(syn_wn)-iniLen)
    return syn_wn


syn_wn=iniSynonym()
syn_wn

In [3]:
findSyn('抛弃'),findSyn('驱逐')

In [4]:
syn_wn['抛弃'],syn_wn['驱逐']

In [5]:
###todo 整理同义词典，把同义词加到情感词典里面并赋值
for k in list(BosonWords.keys()):
    if k in syn_wn:
        synWordList=syn_wn[k]
        for word in synWordList:
            if word not in BosonWords:
                BosonWords[word]=BosonWords[k]

print("The final length of Boson dictionary: ",len(BosonWords))

# 1.3 加载程度副词

# 1.4 给结巴分词加载停用词,情感词,程度副词

In [6]:
import jieba

###stop word
def getStopWords(url):
    with open(url,'r',encoding='utf8') as f:
        stopword = f.readlines()
    return [i.replace('\n','') for i in stopword]

cn_stopwords=getStopWords("NLP/stopwords/cn_stopwords.txt")
baidu_stopword=getStopWords("NLP/stopwords/baidu_stopwords.txt")
hit_stopword=getStopWords("NLP/stopwords/hit_stopwords.txt")
scu_stopword=getStopWords("NLP/stopwords/scu_stopwords.txt")

stopword_list=cn_stopwords+hit_stopword+baidu_stopword+scu_stopword
for stopword in stopword_list:
    jieba.add_word(stopword, freq=None, tag=None)
for k,v in BosonWords.items():
    jieba.add_word(k, freq=None, tag=None)

In [7]:

testSentence=['我今天很开心',
              '我今天不开心',
              '他觉得这个产品一点也不好用',
              '他觉得这个产品']

def getSentiScoreOfSentence(sentence:str)->float:
    ex="///".join(jieba.cut(sentence,cut_all=False))
    cutSentences=ex.split('///')

for sentence in testSentence:
    sentence.split("")

